In [7]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import numpy as np
from my_functions import *

In [8]:
tcurves = load_tcurves(load_filter_tags())

In [2]:
filename = '/home/alberto/almacen/SDSS_spectra_fits/DR16/DR16Q_Superset_v3.fits'
with fits.open(filename) as fread:
    # Criteria in Queiroz et al. 2022
    good_qso = (
        (fread[1].data['ZWARNING'] == 0)
        & (fread[1].data['SN_MEDIAN_ALL'] > 0)
        & (fread[1].data['Z_CONF'] == 3)
        & ((fread[1].data['CLASS_PERSON'] == 3) | (fread[1].data['CLASS_PERSON'] == 30))
    )

    plate = fread[1].data['PLATE'][good_qso]
    mjd = fread[1].data['MJD'][good_qso]
    fiber = fread[1].data['FIBERID'][good_qso]

In [6]:
def JPAS_synth_phot(SEDs, w_Arr, tcurves, which_filters=[]):
    phot_len = len(tcurves['tag'])
    pm = np.zeros(phot_len)    

    if len(which_filters) == 0:
        which_filters = np.arange(phot_len)

    for fil in which_filters:
        w = np.array(tcurves['w'][fil])
        t = np.array(tcurves['t'][fil])

        # Cut w and t where the transmission is grater than some value for
        # performance and bugs
        cut_t_curve = (t > 0.05)
        w = w[cut_t_curve]
        t = t[cut_t_curve]

        sed_interp = np.interp(w, w_Arr, SEDs, left=np.inf, right=np.inf)

        sed_int = np.trapz(w * t * sed_interp, w)
        t_int = np.trapz(w * t, w)
        
        pm[fil] = sed_int / t_int
    return pm[which_filters]

In [3]:
fits_dir = '/home/alberto/almacen/SDSS_spectra_fits/DR16/QSO'

N_src = len(mjd)
pm_SEDs = np.empty((60, N_src))

for src, (pl, mj, fi) in zip(plate, mjd, fiber):
    print(f'{src} / {N_src}')
    spec_name = f'{fits_dir}/spec-{pl}-{mj}-{fi}.fits'

    spec = Table.read(spec_name, hdu=1, format='fits')
    # Correct spec
    spec_w = 10 ** spec['loglam']
    spec_f = spec['flux'] * 1e-17

    # The range of SDSS is 3561-10327 Angstroms. Beyond the range limits,
    # the flux will be 0
    pm_SEDs[:, src] = JPAS_synth_phot(spec_f, spec_w, tcurves)

NameError: name 'fits_dir' is not defined

In [ ]:
# Save the cat
hdr = tcurves['tag'] + ['mjd', 'plate', 'fiber']

pd.DataFrame(
    data=np.hstack(
        pm_SEDs.T,
        mjd.reshape(-1, 1),
        plate.reshape(-1, 1),
        fiber.reshape(-1,1)
    )
).to_csv('csv/J-SPECTRA_QSO_Superset_DR16.csv', header=hdr)